# VAERS Deathrate Analysis

This notebook analyzes the VAERS dataset for the US and compares it to the 2018 (pre-covid) age adjusted death rate.

In [1]:
VAERS_VAX_CSV_SUFFIX = "VAERSVAX.csv"
VAERS_DATA_CSV_SUFFIX = "VAERSDATA.csv"
VAERS_SYMPTOMS_CSV_SUFFIX = "VAERSSYMPTOMS.csv"
EXTENSION = "csv"
CSV_ENCODING = "ISO-8859-1"

# UPDATE THE BELOW VALUE AS A PERCENTAGE BETWEEN 0 and 1 (0 = 0%, 1 = 100%)
# Source: https://www.mayoclinic.org/coronavirus-covid-19/vaccine-tracker
PERCENT_US_VACCINATED = 0.556

# Source https://www.census.gov/quickfacts/fact/table/US/PST045219
US_POPULATION_CURRENT = 331449281

# Source https://www.cdc.gov/nchs/products/databriefs/db355.htm
REFERENCE_2018_DEATH_RATE_PER_100k = 739.1

VAERS_DATA_FOLDER = "./AllVAERSDataCSVS/"

In [2]:
import os

# Enumerate all the CSVs
all_files = [os.path.join(VAERS_DATA_FOLDER, f) for f in os.listdir(VAERS_DATA_FOLDER) if f.lower().endswith(EXTENSION.lower())]

In [3]:
import pandas as pd
import chardet

# Import the CSV's as dataframes.
data = {}
for f in all_files:
    try:
        data[f]  = pd.read_csv(f, encoding=CSV_ENCODING)
    except Exception:
        print(f"Failed to load {f}...")

C:\Users\nuvio\OneDrive - Default Directory\Coding\VAERS Analysis\venv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9,10,11,13,15,16,27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nuvio\OneDrive - Default Directory\Coding\VAERS Analysis\venv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nuvio\OneDrive - Default Directory\Coding\VAERS Analysis\venv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,12,15,23,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nuvio\OneDrive - Default Directory\Coding\VAERS Analysis\venv\lib\site-packages\IPython\core\interactiveshell.py:3444: Dtyp

In [4]:
# Combine all the dataframes, matching them to common VAERS_ID numbers
symptom_dfs = [data[f] for f in data.keys() if f.endswith(VAERS_SYMPTOMS_CSV_SUFFIX)]
symptoms_combined = pd.concat(symptom_dfs)
symptoms_combined = symptoms_combined.set_index('VAERS_ID')

vax_dfs = [data[f] for f in data.keys() if f.endswith(VAERS_VAX_CSV_SUFFIX)]
vax_combined = pd.concat(vax_dfs)
vax_combined = vax_combined.set_index('VAERS_ID')

data_dfs = [data[f] for f in data.keys() if f.endswith(VAERS_DATA_CSV_SUFFIX)]
data_combined = pd.concat(data_dfs)
data_combined = data_combined.set_index('VAERS_ID')

combined = symptoms_combined.merge(vax_combined, how='outer', left_index=True, right_index=True)
combined = combined.merge(data_combined, how='outer', left_index=True, right_index=True)
combined

,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
VAERS_ID,,,,,,,,,,,,,,,,,,,,,
25001,Agitation,8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
25002,Chills,8.1,Convulsion,8.1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Senile dementia, Diabetes mellitus, seizures",~ ()~~~In patient,WAES90040535,1,NaN,NaN,NaN,NaN,NaN
25003,Delirium,8.1,Hypokinesia,8.1,Hypotonia,8.1,NaN,NaN,NaN,NaN,...,NaN,NaN,~ ()~~~In patient,NaN,1,NaN,NaN,NaN,NaN,NaN
25003,Delirium,8.1,Hypokinesia,8.1,Hypotonia,8.1,NaN,NaN,NaN,NaN,...,NaN,NaN,~ ()~~~In patient,NaN,1,NaN,NaN,NaN,NaN,NaN
25004,Chills,8.1,Dermatitis contact,8.1,Oedema genital,8.1,Pelvic pain,8.1,NaN,NaN,...,NaN,NaN,~ ()~~~In patient,890269201,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1708050,Arthralgia,24.1,Burning sensation,24.1,Cold sweat,24.1,Pain,24.1,NaN,NaN,...,NaN,NaN,NaN,NaN,2,09/17/2021,NaN,NaN,NaN,NaN
1708052,Chills,24.1,Fatigue,24.1,Insomnia,24.1,Menstruation irregular,24.1,Night sweats,24.1,...,none,"adenomyosis, chronic sinusitis, arthritis, mig...",NaN,NaN,2,09/17/2021,NaN,NaN,NaN,NaN
1708052,Pain,24.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,none,"adenomyosis, chronic sinusitis, arthritis, mig...",NaN,NaN,2,09/17/2021,NaN,NaN,NaN,NaN


In [5]:
# Get the COVID specific entries.
covid_entries = combined.loc[combined['VAX_TYPE'] == 'COVID19']
covid_entries

,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
VAERS_ID,,,,,,,,,,,,,,,,,,,,,
902418,Hypoaesthesia,24.0,Injection site hypoaesthesia,24.0,NaN,NaN,NaN,NaN,NaN,NaN,...,none,none,NaN,NaN,2,12/15/2020,NaN,NaN,NaN,none
902440,Headache,23.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,12/15/2020,NaN,NaN,NaN,NaN
902446,Erythema,23.1,Feeling hot,23.1,Flushing,23.1,NaN,NaN,NaN,NaN,...,none,"Hypertension, sleep apnea, hypothyroidism",NaN,NaN,2,12/15/2020,NaN,NaN,NaN,"Contrast Dye IV contrast, shellfish, strawberry"
902464,Dizziness,23.1,Electrocardiogram normal,23.1,Hyperhidrosis,23.1,Laboratory test normal,23.1,Presyncope,23.1,...,none,none,NaN,NaN,2,12/15/2020,NaN,NaN,Y,none
902465,Dysgeusia,23.1,Oral pruritus,23.1,Paraesthesia,23.1,Paraesthesia oral,23.1,Parosmia,23.1,...,"Bronchitis, finished prednisone on 12-13-20","hypertension, fibromyalgia",NaN,NaN,2,12/15/2020,NaN,NaN,NaN,Biaxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1708050,Arthralgia,24.1,Burning sensation,24.1,Cold sweat,24.1,Pain,24.1,NaN,NaN,...,NaN,NaN,NaN,NaN,2,09/17/2021,NaN,NaN,NaN,NaN
1708052,Chills,24.1,Fatigue,24.1,Insomnia,24.1,Menstruation irregular,24.1,Night sweats,24.1,...,none,"adenomyosis, chronic sinusitis, arthritis, mig...",NaN,NaN,2,09/17/2021,NaN,NaN,NaN,NaN
1708052,Pain,24.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,none,"adenomyosis, chronic sinusitis, arthritis, mig...",NaN,NaN,2,09/17/2021,NaN,NaN,NaN,NaN


In [6]:
# Get the total number of death and non-death VAERS events.
covid_deaths = covid_entries.loc[covid_entries['DIED'] == 'Y']
covid_non_deaths = covid_entries.loc[covid_entries['DIED'] != 'Y']
display(covid_deaths.shape)
display(covid_non_deaths.shape)
covid_deaths = covid_deaths.shape[0]
covid_deaths

(24580, 51)

(1005855, 51)

24580

In [7]:
# Analyze the death rate against the total vaccinated population and compare to 2018 baseline death rate.
total_fully_vaccinated = int(PERCENT_US_VACCINATED * US_POPULATION_CURRENT)
vaccinated_death_rate_vaers = covid_deaths / total_fully_vaccinated
vaccinated_death_rate_vaers_per_100k = round(vaccinated_death_rate_vaers * 100000, 2)
magnitude_baseline_over_covid = round(REFERENCE_2018_DEATH_RATE_PER_100k/vaccinated_death_rate_vaers_per_100k, 2)
print(f"Of the {total_fully_vaccinated} fully vaccinated americans, {covid_deaths} deaths were reported in VAERS.")
print(f"This yields a {vaccinated_death_rate_vaers_per_100k} deaths per 100k for fully vaccinated individuals in the US.")
print(f"To compare the age-adjusted mortality rate in 2018 (before COVID) in the US was {REFERENCE_2018_DEATH_RATE_PER_100k} per 100k which is {magnitude_baseline_over_covid} times higher than the VAERS reported COVID19 death rate.")

Of the 184285800 fully vaccinated americans, 24580 deaths were reported in VAERS.
This yields a 13.34 deaths per 100k for fully vaccinated individuals in the US.
To compare the age-adjusted mortality rate in 2018 (before COVID) in the US was 739.1 per 100k which is 55.4 times higher than the VAERS reported COVID19 death rate.
